In [4]:
import requests

class Tistory:
    def __init__(self):
        self.defaultParam = {
            'access_token' : '',
            'output' : 'json'
        }
        from secrets import Secret
        self.secret = Secret()
    def getAuthCode(self):
        # this section will remake at frontend. this is poc code
        # change redirectUrl for realservice url later
        authCodeUrl = "https://www.tistory.com/oauth/authorize"
        params = {
            "client_id": self.secret.tistoryAppId,
            "redirect_uri": "http://ykarma1996.tistory.com/",
            "response_type": "code"
        }
        res = requests.get(authCodeUrl, params=params)
        
        # action this url at browser, and get AccessToken manually just now.
        print (res.url)
        
    def getAccessToken(self, authCode="getFromClient"):
        authCode = input("authCode : ") # delete this later
        
        getTokenUrl = "https://www.tistory.com/oauth/access_token"
        params = {
            "client_id": self.secret.tistoryAppId,
            "client_secret": self.secret.tistorySecretKey,
            "redirect_uri": "http://ykarma1996.tistory.com/",
            "code": authCode,
            "grant_type": "authorization_code"
        }
        res  = requests.get(getTokenUrl, params=params)
        self.defaultParam['access_token'] = res.text.split('=')[1]
    
    def getBlogList(self):
        params = self.defaultParam
        getBlogUrl = "https://www.tistory.com/apis/blog/info"
        res = requests.get(getBlogUrl, params=params)
        
        # Object of key: 'blogName' value: 'blogTitle'
        blogList = {}
        for blog in res.json()['tistory']['item']['blogs']:
            blogList[blog['name']] = blog['title']
        
        # Show to FrontEnd
        print (blogList)
        
    def getcatego(self, blogName):
        params = self.defaultParam
        params['blogName'] = blogName
        getcategoUrl = 'https://www.tistory.com/apis/category/list'
        res = requests.get(getcategoUrl, params=params)
        
        # Object of key : 'catego id' value: 'catego name'
        categoList = {}
        for catego in res.json()['tistory']['item']['categories']:
            categoList[catego['id']] = catego['name']
        
        print (categoList)
    
    def postContent(self, blogName, title, content, visibility, category, tag, acceptComment):
        params = self.defaultParam
        params['blogName'] = blogName
        params['title'] = title
        params['content'] = content
        params['visibility'] = visibility
        params['category'] = category
        params['tag'] = tag
        params['acceptComment'] = acceptComment
        
        postUrl = 'https://www.tistory.com/apis/post/write'
        res = requests.post(postUrl, data=params)
        
        print (res.json()['tistory']['url'])


In [5]:
from secrets import Secret
from makeHtml import MakeHtml

secret = Secret()
email = secret.email
apiKeyConfluence = secret.apiKeyConfluence
apiKeyImgbb = secret.apiKeyImgbb
userName = 'geunsam2'

make1 = MakeHtml()
make1.loginProcess(email, apiKeyConfluence, userName, apiKeyImgbb)
make1.getSpaceList()
lists = make1.getContentList("GEUNSAM2")
print (lists)
contentsNum = input('콘텐트 값 : ')
make1.getCententHtml(contentsNum)
make1.rebuildFormat()
make1.rebuildImgStore()
contents = str(make1.htmlSoup)
print ('---')

{'33156': 'CNI와 Docker', '163844': 'Calico?Weave? CNI에 관하여', '229378': 'geunsam2', '884816': 'CNI 동작에 관하여', '1933313': 'Ingress와 LoadBalancer', '7897089': 'gitlab 컨테이너 backup/restore(백업 및 복원)', '8880168': '[confluence to tistory-1]API 조사', '14647297': 'DCOS Marathon-autoscaler and 및 or 옵션 사용', '14778444': '[confluence to tistory-3]tistory api 및 OAuth2.0', '14778479': '[confluence to tistory-2]html 다듬기', '18284545': 'ElasticSearch Index LifeCycle to Template with Fluentbit'}
콘텐트 값 : 14647297
<blockquote><p>본 문서는 DC/OS catalog 서비스에서 제공하는 marathon-autoscaler의 0.5.0 버전을 기준으로 작성되었습니다.</p></blockquote><p>marathon은 kubernetes와 다르게 아직 자체적으로 autoscale에 관한 기능을 공식적으로 지원하지 않는다. 하지만 다행히도 커뮤니티에서 다양한 형태의 오토스케일러들을 만들어서 배포하고 있고, DC/OS 공식 Catalog 에서도 <code>Enterprise</code> 지원은 아니지만, <code>Community</code> 버전으로 autoscaler를 클릭하여 설치할 수 있도록 제공하고 있다. </p><span class="confluence-embedded-file-wrapper image-center-wrapper confluence-embedded-manual-size"><img class="confluence-embedded-image image-center" dat

---


In [6]:
def main():
    ti = Tistory()
    ti.getAuthCode()
    ti.getAccessToken()
    ti.getBlogList()
    
    #categoName = intput()
    blogName = 'ykarma1996'
    ti.getcatego(blogName)
    
    title = input('게시물 제목 : ')
    visibility = 0
    category = input('카테고리 : ')
    tag = input('테그 : ')
    acceptComment = 1
    
    ti.postContent(blogName, title, contents, visibility, category, tag, acceptComment)
    
if __name__ == "__main__":
    main()

https://www.tistory.com/auth/login/?popup=1&redirectUrl=https%3A%2F%2Fwww.tistory.com%2Foauth%2Fauthorize%3Fclient_id%3D1248d0d1dca9f950e085f8cff4bda0f4%26redirect_uri%3Dhttp%253A%252F%252Fykarma1996.tistory.com%252F%26response_type%3Dcode
authCode : ca2c375e32320cbc21f6e3f685b5888403befe78fb357245475fa6c38065f9f794e97e3d
{'ykarma1996': '모두의 근삼이', 'moduboan': '모두의 보안'}
{'1107725': 'Marathon_Mesos', '982641': '리눅스', '1011834': '모두의 보안', '1039593': '모두의 보안 1기', '1184515': '개발 이것저것', '1107726': 'Kubernetes', '1039594': '모두의 보안 2기', '1107727': 'Jenkins', '1089407': 'DevOps', '1039595': '모두의 보안 3기', '1107728': 'Docker', '1107730': '미분류', '1107729': 'Ansible', '1011837': 'SYSTEM', '1107731': 'Database', '1011835': 'WEB', '512220': 'IT정보', '826015': '네트워크', '512216': '블로그', '512317': 'Free Chat', '512221': 'Diary'}
게시물 제목 : DCOS Marathon-autoscaler and 및 or 옵션 사용
카테고리 : 1184515
테그 : dcos,marathon,autoscale,bugfix
https://ykarma1996.tistory.com/116
